# Projeto - Introdução à Computação Quântica

### Desenvolvedores:
* José Rafael Martins Fernandes
* Thiago Hampl Pierri da Rocha

## Introdução

Os *kernel algorithms* desempenham um papel crucial em *machine learning*, permitindo a transformação de dados complexos em espaços de alta dimensão, tornando possível a resolução de problemas não linearmente separáveis. No entanto, o desenvolvimento destes algoritmos utilizando a computação clássica enfrenta desafios significativos quando se trata de lidar com grandes conjuntos de dados e tarefas complexas de *machine learning*. Dessa forma, a computação quântica pode ser uma utilizada como uma maneira de tentar melhorar o desempenho destes algoritmos. Assim, vamos analisar uma pesquisa recente que investiga a interseção entre os s *kernel algorithms* e a computação quântica, com um foco especial na avaliação das possíveis vantagens dos modelos quânticos em tarefas de aprendizado. 

## Revisão do Artigo

No artigo *Power of data in quantum machine learning*, os autores exploram o campo da computação quântica para *machine learning* e buscam avaliar as possíveis vantagens de modelos quânticos em tarefas de aprendizado. Eles começam enfatizando que tarefas de aprendizado de máquina que envolvem dados podem ser significativamente diferentes de outras tarefas computacionais comumente estudadas no domínio da computação quântica. Apesar dessa diferença, os autores demonstram que modelos clássicos de aprendizado de máquina, quando fornecidos com dados, podem competir com modelos quânticos, mesmo em situações em que a complexidade computacional é alta para a computação clássica.

Para avaliar as possíveis vantagens de modelos quânticos em tarefas de aprendizado, os autores introduzem o conceito de uma diferença geométrica, que mede a dissimilaridade entre modelos de *machine learning* clássicos e quânticos com base em suas medidas de similaridade. Eles demonstram como essa diferença geométrica pode ser calculada e usada como uma ferramenta para avaliar o potencial de vantagem quântica na previsão. Se a diferença geométrica for pequena, espera-se que os modelos clássicos de aprendizado de máquina forneçam um desempenho semelhante ou até superior aos modelos quânticos, independentemente dos valores ou rótulos de funções específicas. Por outro lado, uma grande diferença geométrica indica o potencial de uma vantagem quântica.

Assim, os autores realizam experimentos numéricos em conjuntos de dados projetados. Os resultados mostram que os modelos clássicos podem superar os modelos quânticos quando a diferença geométrica é pequena. No entanto, os autores também demonstram que um modelo quântico projetado pode apresentar uma significativa vantagem de previsão sobre os modelos clássicos quando a diferença geométrica é grande. Os experimentos destacam o papel dos dados na determinação das vantagens dos modelos de aprendizado de máquina quântica e enfatizam o potencial dos modelos quânticos projetados em fornecer um aumento na velocidade quântica em tarefas de aprendizado específicas.

## Nosso projeto

O objetivo do projeto consiste em empregar os experimentos mencionados no artigo a fim de verificar ou refutar as conclusões relativas às previsões entre os modelos clássicos e quânticos. Uma vez que o referido artigo aborda as condições que favorecem cada um desses modelos com base na geometria dos dados, planejamos aplicar tanto Kernels Clássicos quanto Quantum Kernels para avaliar tais vantagens, enquanto também examinamos a capacidade de previsão dos modelos quânticos em situações com diferenças significativas na geometria dos dados. Além disso, pretendemos realizar testes em múltiplas bases de dados, a fim de compreender a variabilidade dos resultados e demonstrar o impacto da estrutura dos dados nas conclusões do artigo.

## Problema

O principal desafio que enfrentamos no âmbito deste projeto reside na árdua tarefa de conduzir uma pesquisa para identificar e selecionar as bases de dados mais apropriadas que possam ser empregadas nas Provas de Conceito delineadas no artigo. Adicionalmente, nosso empenho se estende à aplicação de modelos tanto clássicos quanto quânticos, fazendo uso das bibliotecas que dão suporte a esses modelos, notadamente o scikit-learn e o Qiskit, a fim de alcançar resultados robustos e conclusivos em nossa análise.

## Referências
* https://www.nature.com/articles/s41467-021-22539-9
* https://qiskit.org/ecosystem/machine-learning/tutorials/03_quantum_kernel.html
* https://qiskit.org/ecosystem/machine-learning/tutorials/08_quantum_kernel_trainer.html

In [30]:
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

from qiskit_machine_learning.algorithms import QSVC
from qiskit_machine_learning.kernels import FidelityQuantumKernel
from qiskit.primitives import Sampler
from qiskit_algorithms.state_fidelities import ComputeUncompute
from qiskit import Aer
from qiskit.circuit.library import ZZFeatureMap

In [31]:
def classic_SVC(X, y, test_size=0.2, random_state=42):
    # Split the data into train and test sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=random_state)

    # Create an SVC model
    svc_model = SVC(kernel='linear', random_state=random_state)

    # Train the model
    svc_model.fit(X_train, y_train)

    # Make predictions on the test set
    y_pred = svc_model.predict(X_test)

    # Calculate accuracy
    accuracy = accuracy_score(y_test, y_pred)

    return accuracy, y_pred


In [32]:
def quantum_SVC(X, y, test_size=0.2, random_state=42):
    # Split the data into train and test sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=random_state)

    feature_map = ZZFeatureMap(feature_dimension=len(X.columns), reps=2, entanglement="linear")

    sampler = Sampler()

    fidelity = ComputeUncompute(sampler=sampler)

    quantum_kernel = FidelityQuantumKernel(fidelity=fidelity, feature_map=feature_map)

    # Create the QSVC model
    qsvc_model = QSVC(quantum_kernel=quantum_kernel)
    
    # Train the QSVC model
    qsvc_model.fit(X_train, y_train)

    # Test the model
    y_pred = qsvc_model.predict(X_test)

    # Calculate accuracy
    accuracy = accuracy_score(y_test, y_pred)

    return accuracy, y_pred

In [33]:
df = pd.read_csv("data/breast_cancer_data.csv", header = 0)

df.drop('id', axis=1, inplace=True)
df.drop('Unnamed: 32', axis=1, inplace=True)

df['diagnosis'] = df['diagnosis'].map({'M':1,'B':0})
df = df[["radius_mean", "texture_mean", "perimeter_mean", "area_mean", "smoothness_mean", "compactness_mean", "concavity_mean", "symmetry_mean", "fractal_dimension_mean", "diagnosis"]]

X = df.drop("diagnosis", axis=1)
y = df["diagnosis"]

accuracy, y_pred = classic_SVC(X, y)
print(f"Accuracy of the SVC model: {accuracy}")

accuracy_quantum_svc, y_pred_quantum_svc = quantum_SVC(X, y)
print(f"Accuracy of the Quantum SVC model: {accuracy_quantum_svc}")